In [587]:
from __future__ import division
import pandas as pd
import numpy as np
import cPickle as pk
import collections
import editdistance

In [578]:
#t = {}

In [84]:
#Reduce series of lists down to a single list
def flatten2one(xS):
    return [item for sub_list in xS.tolist() for item in sub_list]

In [580]:
#Generate initial uniform distribution
def initial_probs(lis_names):
    vocab_ctr = collections.Counter((flatten2one(lis_names)))
    prob = collections.OrderedDict()
    networds = len(flatten2one(lis_names))
    for k,v in vocab_ctr.items():
        if isinstance(k,float):
            continue
        prob[k]=v/networds
    return prob

In [579]:
#Calculate z(word)
def word_estimation(name,C,B):
    z=dict()
    #t["name"] = name
    #t["B_prob"]= [B[word] for word in name]
    denom=0.0
    fake_news = []
    for w in name:
        try:
            denom+=C[w]/B[w]
        except ZeroDivisionError:
            z[w] = 1
            fake_news.append(w)
    
    for word in name:
        if word in fake_news:
            continue
        #t["last"] = word
        #t["denom"] = denom
        try:
            z[word] = (C[word]/B[word])/denom
        except ZeroDivisionError:
            z[word]=1
    return z

In [149]:
#After getting new z(w), update the parameters/distributions C and B to better represent the data
def update_weights(Z,C,B,networds):
    c=dict()
    b=dict()
    for word in C:
        #print word
        new_c = 0.0
        new_b = 0.0
        for z in Z:
            #print z,
            if word in z:
                #print z[word]
                new_c+=z[word]
                new_b+=(1-z[word])
    
        new_c=new_c/len(Z)
        new_b=new_b/(networds-len(Z))
        #print "new C({}): ".format(word), new_c
        c[word] = new_c
        b[word] = new_b
        #print "next"
    
    #print c
    return (c,b)
    

In [205]:
#Run the EM algorithm that updates the distribution until some degree of convergence
def core_algo1(names):
    #initialize probabilities
    prob_C = prob_B = initial_probs(names)
    networds = len(flatten2one(names))
    for i in range(len(prob_C)):
        #pdb.set_trace()
        all_z = names.apply(word_estimation,args=(prob_C,prob_B))
        prob_C,prob_B = update_weights(all_z,prob_C,prob_B,networds)
    return prob_C,prob_B

In [554]:
#Returns probabilities of being core or background
def assrtcore(word,pofcore):
    return pofcore[word]

def assrtbg(word,pofcore):
    return (1.000001-pofcore[word])

In [581]:
#Calculate edit distance, if (1-edit distance) or how many characters do we retain>75% of the length,
#then we probably have a misspelling, and we can assert that both are the same.
def distmetric(s1,s2,pr1,pr2):
    workons1 = set(s1) - set(s2)
    workons2 = set(s2) - set(s1)
    #Misspellings would be part of the set outside the union
    ret = 0.0
    for w1 in workons1:
        #filter out those words from sentence 2 that are 75% close to this particular word w1 from sentence 1 
        dat = filter(lambda x: (1-editdistance.eval(w1,x)/len(x))>0.75, workons2)
        if len(dat)<1:
            continue
        #The misspelled words are the same, thus both are part of core or part of background
        ret+= reduce( (lambda x,y: x*y),[pr2[a] for a in dat])*pr1[w1] + reduce((lambda x,y: (1-x)*(1-y)),[pr2[a] for a in dat])*(1-pr1[w1])
    
    return ret

In [583]:
#Extra edit functions that can factor into the probability
def edit_func(s1,s2,pr1,pr2):
    fn=0.0
    
    #abbreviation probability 2 word abbrvs and 3 word abbrvs
    dat =( abbrv_func(s1,s2,pr1,pr2,2)+abbrv_func(s1,s2,pr1,pr2,3) )
    fn+=dat
    
    #Probability from misspelling
    fn = fn+distmetric(s1,s2,pr1,pr2)
    return fn

In [586]:
#If I abbreviate Cafe Coffee Day to CCD, they both are still the same, this creates abbreviations and check if same
def abbrv_func(s1,s2,pr1,pr2,n):
    #n corresponds to number of words abbreviations
    ret = 0.0
    for i in range(len(s1)-n+1):
        w1=s1[i:i+n]
        abbs = ''.join([w[:1] for w in w1])
        #If abbreviation exists then those are same, and thus we can say that these words and 
        #their abbreviations are all part of the core or all part of background
        if(abbs in s2):
            ret+= reduce( (lambda x,y: x*y),[pr1[a] for a in w1])*pr2[abbs] + reduce((lambda x,y: (1-x)*(1-y)),[pr1[a] for a in w1])*(1-pr2[abbs])

    return ret

In [585]:
#The core deduplication algorithm
def dedup(sent1,sent2,pofcore1,pofcore2):
    #To be a same match, all core words should be the same and all other words should be background
    fprob = 0.0
    common_w = set(sent1).intersection(sent2)
    s1_uncommon = set(sent1) - set(sent2)
    s2_uncommon = set(sent2) - set(sent1)
    commonprob = 1.0
    #If a word is common, it should be part of the core in both sentences or part of background in both sentences
    for w_ in common_w:
        commonprob*=(assrtcore(w_,pofcore1)*assrtcore(w_,pofcore2) + assrtbg(w_,pofcore1)*assrtbg(w_,pofcore2))
    
    #print "Commons: ", common_w, commonprob
    #If a word is in sentence 1 but not in sentence 2, then it should be a background word
    prob1 = 1.0
    for w1_ in s1_uncommon:
        prob1*=assrtbg(w1_,pofcore1)
    
    #print "uncommon1: ", s1_uncommon, prob1
    
    #If a word is in sentence 2 but not in sentence 1, then it should be a background
    prob2 = 1.0
    for w2_ in s2_uncommon:
        prob2*=assrtbg(w2_,pofcore2)
    
    #print "uncommon2: ", s2_uncommon, prob2
    
    fprob = fprob + prob1*prob2*commonprob
    #print "fprob", fprob
    
    #Add probabilities gotten from edit functions
    fprob= fprob+edit_func(sent1,sent2,pofcore1,pofcore2)
    
    return fprob

In [561]:
#A test function to test my dedup algorithm
def test_sample(alpha,pC,pB):
    #sample_set = pd.Series([["best", "western", "lamplighter", "inn"],["best","western"],["Hotel","Amsterdam"]])
    #pC,pB = core_algo1(sample_set)
    
    pofcore = dict()
    for k,v in pC.items():
        print k,v
        pofcore[k] = alpha*pC[k]/(alpha*pC[k] + (1-alpha)*pB[k])
    p1 =["best", "western","hotel"]
    p2= ["best","western"]
    #print pofcore
    print dedup(p1,p2,pofcore,pofcore)
    

In [588]:
def coreprob(pC,pB, alpha):
    pofcore = dict()
    for k,v in pC.items():
        pofcore[k] = alpha*pC[k]/(alpha*pC[k] + (1-alpha)*pB[k])
    return pofcore

In [577]:
#Load the saved features from prepare_4_EM
def loaddistributions(filename):
    emfeatures = pk.load(open(filename+".pk","rb"))
    alpha = 0.45
    #Working with only london properties as of now
    lon_names = emfeatures.name
    lon_addr = emfeatures.address

    #Bring out the Core and Background distributions from the EM algorithm of name
    pC_distr_name,pB_distr_name = core_algo1(lon_names)
    pC_distr_addr, pB_distr_addr = core_algo1(lon_addr)
    
    #Generate probability of being a core
    pcore_name = coreprob(pC_distr_name,pB_distr_name,alpha)
    pcore_addr = coreprob(pC_distr_addr,pB_distr_addr,alpha)
    
    pk.dump((pcore_name,pcore_addr),open(filename+"distributions.pk","wb"))
    print "File: ", filename+"distributions.pk created"
    return (pcor_name,pcore_addr)

In [589]:
%%time
for i in range(100):
    for j in range(100):
        a=i*j

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 9.75 ms


In [590]:
#EXTRA:

# def preclean(dat):
#     cln_d = set(flatten2one(dat))
    
#     def snip(x,cln):
#         if len(x)<1:
#             return
#         for idx,w in enumerate(x):
#             if len(w)<=2 or w.isdigit():
#                 continue
#             if w[:-1] in cln:
#                 x[idx]=w[:-1]

#     dat.apply(snip,args=(cln_d,))
#     return dat

#Extra function not used, part of old experiments
# def corecmp(w1,w2, pofcore):
#     if(len(w1)==0 or len(w2)==0):
#         return 0
#     if w1[0]==w2[0]:
#         return assrtcore(w1[0], pofcore)*assrtcore(w2[0],pofcore)
#     else:
#         return 0

# def abbrv(w1,w2, pofcore1,pofcore2):
#     if(len(w1)==0 or len(w2)==0):
#         return 0
#     abbs = ''.join([w[:1] for w in w1])
#     print "inside abbrv: ", abbs
#     if abbs==w2[0]:
#         return assrtcore(w1,pofcore1)*assrtcore(w2,pofcore2) + assrtbg(w1,pofcore1)*assrtbg(w2,pofcore2)
#     else:
#         return 0

# def dedup(p1,p2,i,j,pofcore):
#     if(i==len(p1) and j==len(p2)):
#         return 1
#     ret = 0

#     #PI = {corecmp: assrtcore, abbrv: assrtcore, bginsert: assrtbg, bgdelete: assrtbg}
#     E = {corecmp: (1,1), abbrv: (2,1), bginsert: (1,1), bgdelete: (1,2)}
#     for e in E.keys():
#         l,r = E[e] 
#         W1 = p1[i:i+l]
#         W2 = p2[j:j+r]
#         pi = e(W1,W2,pofcore)
#         if pi==0:
#             continue
#         ret+=pi*dedup(p1,p2,i+l,j+r)
    
#     return ret



#lon_names = preclean(lon_names)
#lon_addr = preclean(lon_addr)
#set(flatten2one(lon_names))

# #s1 = ["cafe","coffee","day"]
# #s2= ["cafe","coffee","day"]
# #s2 = ["cafe","coffee","day"]
# s1=["best","western","hotel"]
# s2=["bw","hotel"]
# tempp = {"cafe": 0.5,"coffee": 0.45,"day": 1.0, "ccd": 1.0, "best":0.3,"western": 0.5,"bw":0.70,"hotel":0.24,"coffefe":0.45,"gay":0.8}
# print dedup(s1,s2,tempp,tempp)

#exmpl = pd.Series([["starbucks","coffee"],["kalmane","coffee"],["starbucks"]])

In [591]:
%store?